<a href="https://colab.research.google.com/github/gsalinas01/meme_stock_prices/blob/main/memestockaws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

### Postgres driver

In [67]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-14 00:47:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  5.67MB/s    in 0.2s    

2021-11-14 00:47:35 (5.67 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [68]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("memestockaws").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [69]:
# RDS Database name: memestockdb
# S3 Bucket name: memestockbucket
# (RDS database has already been created)
from pyspark import SparkFiles

## Loading data from S3 bucket into Spark DataFrame

In [71]:
# Read in challenge data from S3 Buckets in AWS ________________________TSLA TWEETS 7 day - Hourly_______________________
# TSLA tweets 7 day - Hourly

url ="https://memestockbucket.s3.amazonaws.com/TSLA_tweet_7days_prior_hist_2021_11_7_perhour.csv"
spark.sparkContext.addFile(url)
tweets_7day_hourly = spark.read.csv(SparkFiles.get("TSLA_tweet_7days_prior_hist_2021_11_7_perhour.csv"), sep=",", header=True, inferSchema=True)
tweets_7day_hourly.show(truncate=False)

+---+----------+------------+-------------------+
|_c0|Unnamed: 0|tweet_counts|date_UTC           |
+---+----------+------------+-------------------+
|0  |0         |22          |2021-10-31 18:00:00|
|1  |1         |30          |2021-10-31 19:00:00|
|2  |2         |16          |2021-10-31 20:00:00|
|3  |3         |34          |2021-10-31 21:00:00|
|4  |4         |93          |2021-10-31 22:00:00|
|5  |5         |17          |2021-10-31 23:00:00|
|6  |6         |18          |2021-10-31 00:00:00|
|7  |7         |31          |2021-11-01 01:00:00|
|8  |8         |49          |2021-11-01 02:00:00|
|9  |9         |35          |2021-11-01 03:00:00|
|10 |10        |46          |2021-11-01 04:00:00|
|11 |11        |22          |2021-11-01 05:00:00|
|12 |12        |9           |2021-11-01 06:00:00|
|13 |13        |12          |2021-11-01 07:00:00|
|14 |14        |27          |2021-11-01 08:00:00|
|15 |15        |31          |2021-11-01 09:00:00|
|16 |16        |52          |2021-11-01 10:00:00|


In [73]:
# Read in challenge data from S3 Buckets in AWS _____________________TSLA STOCK HISTORY 7 day - Hourly_______________________________
# TSLA Stock History 7 day - Hourly
url2 ="https://memestockbucket.s3.amazonaws.com/TSLA_stock_hist_hourly_2021_11_7.csv"
spark.sparkContext.addFile(url2)
tsla_stock_hourly = spark.read.csv(SparkFiles.get("TSLA_stock_hist_hourly_2021_11_7.csv"), sep=",", header=True, inferSchema=True)
tsla_stock_hourly.show(truncate=False)

+---+-------------------------+------------------+------------------+------------------+------------------+------------------+--------+---------------------+-------------------+
|_c0|Unnamed: 0               |Open              |High              |Low               |Close             |Adj Close         |Volume  |Percent_Day_Change   |date_UTC           |
+---+-------------------------+------------------+------------------+------------------+------------------+------------------+--------+---------------------+-------------------+
|0  |2021-10-28 09:30:00-04:00|1068.3050537109377|1081.0            |1055.2099609375   |1060.3599853515623|1060.3599853515623|8788599 |-0.7437078324937674  |2021-10-28 14:00:00|
|1  |2021-10-28 10:30:00-04:00|1060.1800537109377|1077.4649658203123|1058.8900146484377|1070.250732421875 |1070.250732421875 |4857422 |0.9499026769733244   |2021-10-28 15:00:00|
|2  |2021-10-28 11:30:00-04:00|1070.83984375     |1079.68994140625  |1064.3499755859377|1076.3800048828123|107

In [74]:
# Read in challenge data from S3 Buckets in AWS _____________________SPY History 7 day - Hourly_______________________________
# SPY History 7 day - Hourly
url34 ="https://memestockbucket.s3.amazonaws.com/SPY_stock_hist_hourly_2021_11_7.csv"
spark.sparkContext.addFile(url34)
SPY_7_day_hourly = spark.read.csv(SparkFiles.get("SPY_stock_hist_hourly_2021_11_7.csv"), sep=",", header=True, inferSchema=True)
SPY_7_day_hourly.show(truncate=False)

+---+-------------------------+------------------+------------------+------------------+------------------+------------------+--------+---------------------+-------------------+
|_c0|Unnamed: 0               |Open              |High              |Low               |Close             |Adj Close         |Volume  |Percent_Day_Change   |date_UTC           |
+---+-------------------------+------------------+------------------+------------------+------------------+------------------+--------+---------------------+-------------------+
|0  |2021-10-28 09:30:00-04:00|455.8999938964844 |457.125           |455.68011474609375|457.05999755859375|457.05999755859375|9880982 |0.2544425702213804   |2021-10-28 14:00:00|
|1  |2021-10-28 10:30:00-04:00|457.0700073242188 |457.97601318359375|457.0             |457.5199890136719 |457.5199890136719 |5874814 |0.09844918332913012  |2021-10-28 15:00:00|
|2  |2021-10-28 11:30:00-04:00|457.5299987792969 |457.95001220703125|456.8599853515625 |457.2999877929688 |457